In [14]:
import boto3
import json

In [8]:
#connect to aws

session = boto3.Session( aws_access_key_id,
       aws_secret_access_key,
       region_name=region_name)
kinesis_client = boto3.client("kinesis", region_name="ca-central-1")


In [10]:
#create data stream
response = kinesis_client.create_stream(StreamName = "tweeter_stream", ShardCount=1)

print(response)

{'ResponseMetadata': {'RequestId': 'd7192f76-b458-cf7a-b73d-57b9a682d232', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd7192f76-b458-cf7a-b73d-57b9a682d232', 'x-amz-id-2': 'Bn/x5CIYwQEyXPhOYK1yyPoF7Lv7H2J5UUitmjMtJuDgmUnbJC0e42qwrjuXj0ruNVKYx1BcVUFOvL0PDmeZ1kS+f8GOJUhR', 'date': 'Mon, 10 Feb 2025 21:58:39 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [19]:
data = {"tweet":"i am so happy"}
jdata = json.dumps(data).encode("utf-8")
print(jdata)

b'{"tweet": "i am so happy"}'


In [20]:
response = kinesis_client.put_record(StreamName = "tweeter_stream", Data = jdata,PartitionKey="partition-1" )

In [22]:
#get records
#1)get shard details 

response = kinesis_client.describe_stream(StreamName="tweeter_stream")

#2)get shard id
shard_id = response["StreamDescription"]["Shards"][0]["ShardId"]

#3)get shard iterator
response = kinesis_client.get_shard_iterator(
    StreamName="tweeter_stream",
    ShardId=shard_id,
    ShardIteratorType="TRIM_HORIZON",  # or "LATEST"
)

shard_iterator = response["ShardIterator"]

#4)fetch records
response = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=10)

# Extract records
records = response["Records"]
for record in records:
    print(record["Data"].decode("utf-8"))  # Decode JSON string back to dict if needed



{"tweet": "i am so happy"}
